In [ ]:
import numpy as np
import matplotlib.pyplot as plt


Lx, Ly = 15000, 15000
nx, ny, nz = 20,20,15
max_depth = 2500


x_edges = np.linspace(0, Lx, nx + 1)
y_edges = np.linspace(0, Ly, ny + 1)
z_edges = np.linspace(0, max_depth, nz + 1)

xc = 0.5 * (x_edges[:-1] + x_edges[1:])
yc = 0.5 * (y_edges[:-1] + y_edges[1:])
zc = 0.5 * (z_edges[:-1] + z_edges[1:])  

Xc2d, Yc2d = np.meshgrid(xc, yc, indexing='ij')


R = np.sqrt((Xc2d - Lx / 2)**2 + (Yc2d - Ly / 2)**2)
Rmax = Lx / 2
h0 = max_depth
basin_depth = h0 * (1 - (R / Rmax)**2)
basin_depth[R > Rmax] = 0  

rho_top = 1500.0
rho_bot = 3500.0


rho_model = np.full((nx, ny, nz), 3500.0) 

for k, z in enumerate(zc):  
    rho_z = rho_top + (rho_bot - rho_top) * (z / max_depth)**2
    inside = z < basin_depth
    rho_model[:, :, k][inside] = rho_z

ix = nx // 2
plt.figure(figsize=(8, 4))
plt.imshow(rho_model[ix, :, :].T, origin='upper',
           extent=[y_edges[0], y_edges[-1], z_edges[-1], z_edges[0]],
           cmap='jet', aspect='auto')
plt.colorbar(label="Density (kg/m³)")
plt.title(f"Truly Horizontal Layers at X = {xc[ix]:.0f} m")
plt.xlabel("Y (m)")
plt.ylabel("Depth (m)")
plt.tight_layout()
plt.show()


In [ ]:

from gravity3d_variable_density import compute_gravity

x_obs = np.linspace(0, Lx,15)
y_obs = np.linspace(0, Ly,15)
Xobs, Yobs = np.meshgrid(x_obs, y_obs, indexing='ij')
Zobs = np.zeros_like(Xobs) 


rho_contrast = rho_model - 3500.0 


gz = compute_gravity(Xobs, Yobs, Zobs, x_edges, y_edges, z_edges, rho_contrast)

noise_level = 0.01 * np.std(gz)
gz += np.random.normal(0, noise_level, size=gz.shape)

In [ ]:
from scipy.optimize import differential_evolution
from scipy.interpolate import RectBivariateSpline
import matplotlib.pyplot as plt

n_ctrl_x, n_ctrl_y = 5, 5
x_ctrl = np.linspace(0, Lx, n_ctrl_x)
y_ctrl = np.linspace(0, Ly, n_ctrl_y)


def interpolate_basin_spline(control_depths):
    depth_grid = control_depths.reshape((n_ctrl_x, n_ctrl_y))
    spline = RectBivariateSpline(x_ctrl, y_ctrl, depth_grid, kx=3, ky=3)
    basin_surface = spline(xc, yc) 
    return np.clip(basin_surface, 0, max_depth)

# Objective (Misfit) Function 
def misfit_spline(control_depths):
    basin_depth = interpolate_basin_spline(control_depths)
    

    rho_model = np.full((nx, ny, nz), 3500.0)
    for k, z in enumerate(zc):
        rho_z = rho_top + (rho_bot - rho_top) * (z / max_depth)**2  
        rho_model[:, :, k] = rho_z

    rho_contrast = rho_model - 3500.0
    gz_pred = compute_gravity(Xobs, Yobs, Zobs, x_edges, y_edges, z_edges, rho_contrast)
    return np.linalg.norm(gz_pred - gz)


iteration = {'count': 0}
def callback_spline(xk, convergence):
    iteration['count'] += 1
    current_misfit = misfit_spline(xk)
    print(f"Iteration {iteration['count']:02d}: Misfit = {current_misfit:.4f}")


num_params = n_ctrl_x * n_ctrl_y
bounds = [(0, max_depth)] * num_params

result_spline = differential_evolution(
    misfit_spline,
    bounds,
    strategy='randtobest1bin',
    maxiter=70,
    popsize=20,
    mutation=(0.5, 1.0),
    recombination=0.9,
    callback=callback_spline,
    polish=True,
    disp=False
)

best_depths = result_spline.x
depth_grid = best_depths.reshape((n_ctrl_x, n_ctrl_y))
print("\nInversion completed.\nRecovered control point depths (m):")
print(depth_grid)


recovered_basin = interpolate_basin_spline(best_depths)
